# Meteorology Data Extraction

In this notebook, we will:
1. Run code to associate sample plots (fields) with their nearest GridMET cell.
2. Use a simple Python API that downloads GridMET data from the THREDDS server.

